## Part One-Scraping Wikipedia to Build the Dataframe

### Downloading the libraries needed dependencies needed for the assingment

In [1]:
import numpy as np 
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', None)

#max_info_rows is the maximum number of rows for which a frame will perform a null check on its columns when.
#A value of None means always perform a null check when repr'ing.

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes #geopy makes it easy for Python developers to locate the coordinates of addresses, cities, countries, and landmarks across the globe using third-party geocoders Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

#Beautiful Soup
from bs4 import BeautifulSoup

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # install foluim 
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


### Reading Table Info from Wikipedia

In [2]:
wikiresults = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(wikiresults,'lxml')

In [ ]:
table = soup.find( 'table', {'class':'wikitable sortable'})  #find table and print table 
#print (wiki_table)

In [ ]:
rows= table.find_all('tr') ##tr=talble rows
#print(rows)
column_names= [v.text.replace('\n', '') for v in rows[0].find_all('th')]  #th=table header
print(column_names)

In [ ]:
tabledata = soup.find('table')
colvals = tabledata.find_all('td')    #td=table data
elem_cnt = len(colvals)
postcode = []
borough = []
neighborhood = []

for i in range(0, elem_cnt, 3):
        postcode.append(colvals[i].text.strip())
        borough.append(colvals[i+1].text.strip())
        neighborhood.append(colvals[i+2].text.strip())

### Building the dataframe

In [ ]:
df = pd.DataFrame(data=[postcode, borough, neighborhood]).transpose()
df.columns = ['Postcode', 'Borough', 'Neighborhood']
df.head()

### Cleaning the data

In [8]:
df.drop(df[df['Borough'] == 'Not assigned'].index, inplace=True)
df.loc[df.Neighborhood == 'Not assigned', "Neighborhood"] = df.Borough
df_grouped = df.groupby(['Postcode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
df_grouped.columns = ['Postcode', 'Borough', 'Neighborhood']
df_grouped.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [9]:
df_grouped.shape

(103, 3)

## Part Two-  Building the DataFrame

### Downloading and Reading the Geospatial data

In [10]:
df2 = pd.read_csv('http://cocl.us/Geospatial_data')
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
df2.columns = ['Postcode', 'Latitude', 'Longitude']  
dfcombined = pd.merge(df_grouped, df2, on=['Postcode'], how='inner')  ##Merging the two dataframes
dfcombined.shape
dfcombined.head(12)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


## Part 3- Exploring and Clustering in Canada

In [22]:
neighborhoods = dfcombined[['Borough', 'Neighborhood', 'Latitude', 'Longitude']]
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,Scarborough,Woburn,43.770992,-79.216917
4,Scarborough,Cedarbrae,43.773136,-79.239476


In [23]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [25]:
neighborhoods['Borough'].unique()


array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       "Queen's Park", 'Mississauga', 'Etobicoke'], dtype=object)

### Segmenting and cluster only the neighborhoods in Toronto 

In [26]:
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


### MAP OF TORONTO

In [27]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(dfcombined['Latitude'], dfcombined['Longitude'], dfcombined['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto